In [ ]:
# https://github.com/Mozilla-Ocho/llamafile/blob/main/llama.cpp/server/README.md#api-endpoints

In [2]:
import requests
import os
from pathlib import Path

In [16]:
# File name and URL
file_name = "llava-v1.5-7b-q4.llamafile"
url = (
    "https://huggingface.co/Mozilla/llava-v1.5-7b-llamafile/resolve/main/llava-v1.5-7b-q4.llamafile?download=true"
)

# Check if file is present in the current directory
if not os.path.exists(file_name):
    print(f"'{file_name}' not found. Downloading...")
    # Download the file
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for HTTP errors
    with open(file_name, "wb") as f:
        f.write(response.content)
    print(f"'{file_name}' has been downloaded.")
else:
    print(f"'{file_name}' already exists in the current directory.")

# Add the LLM downloaded from HF
model_path = file_name

'llava-v1.5-7b-q4.llamafile' already exists in the current directory.


In [ ]:
# ./llava-v1.5-7b-q4.llamafile --server --nobrowser
# chmod +x llava-v1.5-7b-q4.llamafile
# 

In [6]:
from langchain_community.retrievers import PubMedRetriever
retriever = PubMedRetriever(MAX_QUERY_LENGTH=5)
retriever.invoke('carcinoma')

[Document(page_content='BACKGROUND: Apoptosis can fuel oncogenesis by the education of surrounding stromal cells. However, the function of cancer-associated fibroblasts (CAFs), which interacted with apoptotic cancer cells, in oral squamous cell carcinoma (OSCC) progression is still unknown.\nOBJECTIVES: This study aimed to explore the prognostic value of apoptosis and the biological effects of CAFs, interacted with apoptotic cancer cells, on OSCC.\nMETHODS: A total of 166 samples from OSCC patients were stained via TUNEL reaction to evaluate the correlation between apoptosis and clinical characteristics. Cell viability and proliferation were assessed through flow cytometry and CCK-8 assays, respectively. Levels of mRNA and protein were examined through qRT-PCR, western blot and immunofluorescence.\nRESULTS: Higher percentage of apoptotic cancer cells in OSCC positively correlated with more Ki67 cells and predicted poor clinical outcomes. Conditioned medium from CAFs exposed to apoptoti

In [4]:
chain_rag = chain.with_types(input_type=Question)


In [5]:
from langserve.client import RemoteRunnable

runnable = RemoteRunnable("http://localhost:8000/llamafile")

              Cancer is a disease characterized by the uncontrolled growth and spread of abnormal cells. Apoptosis, or programmed cell death, is a natural process that can occur in normal cells to maintain tissue homeostasis, but it can also play a role in oncogenesis by education of surrounding stromal cells. Cancer-associated fibroblasts (CAFs) interact with apoptotic cancer cells and can promote the progression of oral squamous cell carcinoma (OSCC). This study aimed to explore the prognostic value of apoptosis and the biological effects of CAFs on OSCC. The researchers analyzed 166 samples from OSCC patients using TUNEL reaction, flow cytometry, qRT-PCR, western blot, and immunofluorescence to evaluate the correlation between apoptosis and clinical characteristics, cell viability and proliferation, mRNA and protein levels. They found that higher percentage of apoptotic cancer cells in OSCC positively correlated with more Ki67 cells and predicted poor clinical outcomes. Conditioned 

In [ ]:
from langchain_community.llms.llamafile import Llamafile
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.retrievers import PubMedRetriever
from langchain_core.pydantic_v1 import BaseModel

llm = Llamafile()
retriever = PubMedRetriever(MAX_QUERY_LENGTH=10)

template = """Based on the question answer me.
              Context:{context}
              Question: {question}
              Answer:
"""  # noqa: E501

prompt = ChatPromptTemplate.from_template(template)

chain = ({"context":retriever,"question":RunnablePassthrough()}
                   | prompt
                   | llm
                   | StrOutputParser()
)

# Add typing for input
class Question(BaseModel):
    __root__: str


chain_rag = chain.with_types(input_type=Question)
